TO DO: 

- Run gemma on vast.ai
- modify the embedding_analysis classes in order to work on gemma structure
- test the gemma IT as well "google/gemma-2-2b-it"


In [4]:
# filepath: vscode-notebook-cell:/home/ailab/Code/MechInt/feature-circuits/experiments/semantic_dispersion.ipynb#Y103sZmlsZQ%3D%3D
import sys
sys.path.append('/home/ailab/Code/MechInt/feature-circuits/experiments')

from embedding_analysis import BlocksEmbeddingAnalysis
from embedding_analysis import EmbeddingAnalysis
import nnsight
from nnsight import NNsight
from nnsight import LanguageModel

device = 'cuda:0'
gemma = LanguageModel("google/gemma-2-2b", device_map="cuda")#, device_map="auto")
print(gemma)



# # Example usage of the class
# text = "Your input text here"
# analysis = BlocksEmbeddingAnalysis(llm, text)
# analysis.analyze_through_blocks()

Gemma2ForCausalLM(
  (model): Gemma2Model(
    (embed_tokens): Embedding(256000, 2304, padding_idx=0)
    (layers): ModuleList(
      (0-25): 26 x Gemma2DecoderLayer(
        (self_attn): Gemma2Attention(
          (q_proj): Linear(in_features=2304, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (v_proj): Linear(in_features=2304, out_features=1024, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2304, bias=False)
        )
        (mlp): Gemma2MLP(
          (gate_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (up_proj): Linear(in_features=2304, out_features=9216, bias=False)
          (down_proj): Linear(in_features=9216, out_features=2304, bias=False)
          (act_fn): PytorchGELUTanh()
        )
        (input_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (post_attention_layernorm): Gemma2RMSNorm((2304,), eps=1e-06)
        (pre_feedforward_layernorm): Gemm

In [ ]:
# Input string
text = "The Eiffel Tower is in the city of"
# High variance text
high_variance_text = """The quantum mechanics lecture covered topics from wave-particle duality 
to Schrödinger's cat, while the culinary class next door was busy preparing a variety of exotic dishes."""

# Low variance text
low_variance_text = """The cat sat on the mat, purring softly as it basked in the warm sunlight streaming 
through the window."""



print(f"Number of tokens in text: {len(gemma.tokenizer.encode(text))}")

print(f"Number of tokens in high variance text: {len(gemma.tokenizer.encode(high_variance_text))}")
print(f"Number of tokens in low variance text: {len(gemma.tokenizer.encode(low_variance_text))}")

Number of GPT tokens in text: 9
Number of GPT tokens in high variance text: 35
Number of GPT tokens in low variance text: 26


In [6]:
with gemma.trace(text):

    # Access the last layer using h[-1] as it's a ModuleList
    # Access the first index of .output as that's where the hidden states are.

    # token_embed = llm.transformer.wte.output.save()    
    # pos_embed = llm.transformer.wpe.output.save()
    # input_embed = llm.transformer.drop.input.save()
    # #llm.transformer.h[-1].mlp.output[0][:] = 0
    # ln_0 = llm.transformer.h[0].ln_1.output.save()
    # #attn_0 = llm.transformer.h[0].attn.output.save()

    # # Access the residual stream after the MLP of the 3rd GPT block
    # residual_after_mlp_3 = llm.transformer.h[2].mlp.output.save()

    # Logits come out of model.lm_head and we apply argmax to get the predicted token ids.
    token_ids = gemma.lm_head.output.argmax(dim=-1).save()

print("\nToken IDs:", token_ids)
#print("type:", type(token_embed))

# Apply the tokenizer to decode the ids into words after the tracing context.
print("Prediction:", gemma.tokenizer.decode(token_ids[0][-1]))
#print("Residual after MLP of 3rd GPT block:", residual_after_mlp_3.shape)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

NNsightError: CUDA out of memory. Tried to allocate 82.00 MiB. GPU 0 has a total capacity of 5.92 GiB of which 40.81 MiB is free. Including non-PyTorch memory, this process has 5.17 GiB memory in use. Process 161104 has 724.00 MiB memory in use. Of the allocated memory 5.03 GiB is allocated by PyTorch, and 62.00 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)